# 6강) FAISS 실습

### Requirements

In [13]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.4.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1
!pip install faiss-cpu > /dev/null 2>&1

In [14]:
import torch
import random
from pprint import pprint

In [9]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx" -O encoders.tar.gz && rm -rf /tmp/cookies.txt

--2022-07-25 08:33:51--  https://docs.google.com/uc?export=download&confirm=t&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx
Resolving docs.google.com (docs.google.com)... 142.250.152.139, 142.250.152.113, 142.250.152.101, ...
Connecting to docs.google.com (docs.google.com)|142.250.152.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jodoj2jbrsspbr97oriqftka1m1hmq0i/1658738025000/10000090644219567406/*/11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx?e=download&uuid=5aeb13e0-d399-4355-a766-fe73b2848395 [following]
--2022-07-25 08:33:51--  https://doc-04-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jodoj2jbrsspbr97oriqftka1m1hmq0i/1658738025000/10000090644219567406/*/11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx?e=download&uuid=5aeb13e0-d399-4355-a766-fe73b2848395
Resolving doc-04-3s-docs.googleusercontent.com (doc-04-3s-docs.googleusercontent.com)... 142.250.148.

In [10]:
!tar -xf ./encoders.tar.gz

In [11]:
# # 1. 미리 학습해둔 encoder file 다운로드
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq" -O dense_encoder.tar.gz && rm -rf /tmp/cookies.txt

# # 2. .tar.gz file 압축해제 
# !tar -xf ./dense_encoder.tar.gz

## [과제 1] Passage retrieval 준비하기

In [16]:
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

# TODO: load a model trained at Daily Mission 4

model_checkpoint = "bert-base-multilingual-cased"
p_encoder = BertEncoder.from_pretrained("./p_encoder/").cuda()
q_encoder = BertEncoder.from_pretrained("./q_encoder/").cuda()

# You can use "bert-base-multilingual-cased" for tokenizer, if you did not save tokenizer at Daily Mission 4.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Search corpus: KorQuAD validation context

In [17]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7. Subsequent calls will reuse this data.


9606

In [18]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [19]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

In [20]:
import numpy as np
p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)

(960, 768)

Question encoder를 활용해여 question dense embedding 생성

In [21]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은?',
 '금강산의 겨울 이름은?',
 '유관순 열사는 당시 어떤 종교를 믿고 있었는가?',
 '1997년 10월 23일, 국회 본회의 대표 연설에서 전두환, 노태우 전 대통령에 대한 사면을 촉구한 새정치 국민회의 의원은?',
 '셰르징거가 찾아왔다가 우연히 푸시캣 돌스에 영입된 곳은?']

In [22]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [23]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [24]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[806, 401, 872,  ...,  20, 216, 745],
        [367, 170, 939,  ..., 668, 852, 413],
        [523, 952, 548,  ..., 273, 849,  42],
        [ 35, 914, 398,  ..., 569, 318,  76],
        [188, 346, 210,  ..., 273, 267, 681]], device='cuda:0')
--- 0.020125627517700195 seconds ---


In [25]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with score 57.2859
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [26]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)


In [27]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [28]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0030257701873779297 seconds ---


In [29]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[169.18578 170.90515 171.20724 171.42766 171.88556]
 [141.62924 159.40785 163.65308 163.87369 164.61594]
 [133.53859 136.49117 136.66232 136.72562 136.8421 ]
 [135.6503  141.44838 141.93663 142.38736 142.677  ]
 [204.16061 206.21935 209.82663 210.6087  212.82346]]


=======[Index of Top-5 Passages]=======
[[118 128 255 369 786]
 [452 498 344 500 150]
 [952 498 133 183 548]
 [118 482 631 277 614]
 [145 697 541 848 280]]


In [30]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with distance 169.1858
2015년 2월, 나경원 새누리당 의원이 국회 외교통일위원장에 선출됐다. 국회 외통위원장에 여성 의원이 선출된 것은 헌정사상 처음이다. 국회는 26일 본회의에서 보궐선거를 진행해 208표 중 176표를 얻은 나경원 의원을 새로운 국회 외통위원장에 선출했다고 밝혔다. 나 의원은 당선 인사에서 "우리 외교가 녹록치 않다. 정부와 국회가 소통과 협업을 통해 여러 가지 산적한 현안들을 풀어가겠다"면서 "남북관계를 풀어 나가서 통일의 초석을 놓는데 국회가 앞장서도록 노력하겠다. 여야가 머리를 맞대서 10년간 미뤄온 북한인권법도 반드시 마무리 할 수 있도록 노력하겠다"고 밝혔다. 나 의원은 앞서 이날 오전 새누리당 의원총회에서 진행된 외통위원장 후보 경선에서 총 135표 가운데 92표를 얻어 43표 득표에 그친, 친한 동료 의원이자 라이벌인

## [과제 2] Wikipedia documents에 대해 FAISS retrieval 실습해보기

In [31]:
# 3. 미리 올려둔 wiki docs 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

--2022-07-25 08:46:04--  https://docs.google.com/uc?export=download&confirm=t&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF
Resolving docs.google.com (docs.google.com)... 142.250.152.101, 142.250.152.102, 142.250.152.113, ...
Connecting to docs.google.com (docs.google.com)|142.250.152.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7e3h9n8gta87atccr6k977ckl3c4flfd/1658738700000/10000090644219567406/*/1eD57pCCvhiXLrHBb1j07QX1VQQDogejF?e=download&uuid=9e8a06fc-9456-4d37-a58c-0773fe8ae833 [following]
--2022-07-25 08:46:04--  https://doc-0c-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7e3h9n8gta87atccr6k977ckl3c4flfd/1658738700000/10000090644219567406/*/1eD57pCCvhiXLrHBb1j07QX1VQQDogejF?e=download&uuid=9e8a06fc-9456-4d37-a58c-0773fe8ae833
Resolving doc-0c-3s-docs.googleusercontent.com (doc-0c-3s-docs.googleusercontent.com)... 142.250.148.

In [32]:
# First load wikipedia dump
import json

# TODO: Write your own path and load json file
dump_path = "./wikipedia_documents.json"
with open(dump_path, 'r', encoding='utf-8') as f:
    wiki = json.load(f)

In [33]:
corpus = [dic['text'] for dic in wiki.values()]
len(corpus)

60613

In [34]:
corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [35]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
wiki_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    wiki_embs.extend(outputs)

import numpy as np
wiki_embs = np.array(wiki_embs)
wiki_embs.shape  # (num_passage, emb_dim)

(60613, 768)

In [36]:
# TODO: build faiss indexer using your own parameters
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = wiki_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(wiki_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(wiki_embs)
indexer.add(wiki_embs)

In [37]:
# TODO: transform query to dense vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'

valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

q_embs.shape  # (num_query, emb_dim)

(1, 768)

In [38]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.013764381408691406 seconds ---


In [39]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[169.08406 169.14308 170.47746 171.16904 171.28467]]


=======[Index of Top-5 Passages]=======
[[59364 48128 22261 35509 40764]]


In [40]:
print("[Search query]\n", q, "\n")
# 주의: 우리는 wiki 문서 전체에서 임의의 질문을 물어보는 것이므로 ground truth 가 존재하지 않습니다

d = D[0]
i = I[0]
for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(corpus[i[j]])
    print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

Top-1 passage with distance 169.0841
국내 산업 회복 법령이 비헌법적이란 것을 미국 대법원이 판결을 내린지 단 5주 후에 와그너 상원은 성공적으로 의회를 통하여 노동법에 관한 새로운 일을 하였다. 1935년 7월 5일 의회는 국내 노동 관계법을 통과시켜 조합들로 결성하는 데 근로자들의 권리를 거듭 단언하였다. 와그너 상원의 이름을 따 더욱 일반적으로 와그너 법령으로 불린 국내 노동 관계법은 미국 역사상 노동법의 가장 중요한 표본이다. 법령은 조합에 가입하기 위하여 단순하게 직원들을 해고시키는 것으로부터 고용주들을 금지한다. 그것은 또한 조합 활동들에 방해, 조합과 협상 거부 혹은 노동 조직에 통제를 얻는 것을 포함한 불공평한 비지니스 관행들로부터 회사들을 금지하기도 한다. 법령은 또한 조합에 가입하는 데 요구된 근로자들로부터 조합들을 금한다. 이 조항들을 집행하는 데 와그너 법령은 회원들이 5년의 기간을 위하여 대통령에 의하여 임명된 국내 노동 관계 위원회를 창조하였다. 예를 들어 회사가 법령을 위반할 때 국내 노동 관계 위원회는 고용주가 의심스러운 활동을 중단한 것을 요구한 정지 명령을 내릴 수 있다. 만약 회사가 고집한다면 부당 관행을 중지하는 데 회사에 강제 판결을 위하여 국내 노동 관계 위원회가 미국 연방 항소법원으로 갈 수 있다. 국내 노동 관계 위윈회는 조합을 형성하거나 아니면 조합이 형성되었다는 것을 결정하는 데 근로자들이 자신들의 대표들을 결심시키는 선거들을 치르는 도움을 주면서 조합들을 흥행한다.\n\n그는 국내 산업 회복 법령의 3개의 규정들에서 각별히 흥미를 가졌으며 독립적인 노동 조합들을 통하여 향상된 근무 상태들을 위하여 단체 교섭을 하는 데 근로자들의 권리, 최저 임금, 최대 시간과 근무 상태들을 위한 산업 규약의 설립과 실업자들을 위한 직업들을 창조하는 데 공공사업진흥국의 설립이었다